## **Install Required Python Libraries**

You are editing this notebook, so we assume you have already cloned the repository from  
[https://github.com/klinucsd/interagency-tracking-system](https://github.com/klinucsd/interagency-tracking-system) into the folder ~/work/interagency-tracking-system. Please verify that the `requirements.txt` file is available in that folder before proceeding.

In [1]:
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


## **Unzip Reference Data in GDB format**

In [2]:
! unzip -o a_Reference.gdb.zip 

Archive:  a_Reference.gdb.zip
  inflating: a_Reference.gdb/a00000007.CatRelTypesByBackwardLabel.atx  
  inflating: a_Reference.gdb/a0000000c.horizon  
  inflating: a_Reference.gdb/a0000000c.gdbtablx  
  inflating: a_Reference.gdb/a00000007.gdbtablx  
  inflating: a_Reference.gdb/timestamps  
  inflating: a_Reference.gdb/a00000010.spx  
  inflating: a_Reference.gdb/a00000011.I36Original_Acti.atx  
  inflating: a_Reference.gdb/a00000006.FDO_UUID.atx  
  inflating: a_Reference.gdb/a00000007.gdbtable  
  inflating: a_Reference.gdb/a00000004.gdbtablx  
  inflating: a_Reference.gdb/a00000010.freelist  
  inflating: a_Reference.gdb/a0000000b.gdbtablx  
  inflating: a_Reference.gdb/a00000001.gdbindexes  
  inflating: a_Reference.gdb/a00000009.horizon  
  inflating: a_Reference.gdb/a0000000b.gdbtable  
  inflating: a_Reference.gdb/a00000011.gdbtable  
  inflating: a_Reference.gdb/a0000000a.horizon  
  inflating: a_Reference.gdb/a00000003.gdbtable  
  inflating: a_Reference.gdb/a00000010.horizon

## **Download Timber Industry data**

In [3]:
from utils.download_from_box import download_box_shared_link

timber_industry_box_link = 'https://calfire.box.com/s/41d807br6stus8jxbsd528sdil5d2xu4'
download_box_shared_link(timber_industry_box_link)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: FFSC_MOU_v2_IndustryOnly_20250401.zip
    ✓ Saved: FFSC_MOU_v2_IndustryOnly_20250401.zip (477,218,163 bytes)



['./FFSC_MOU_v2_IndustryOnly_20250401.zip']

## **Unzip Timber Industry data**

In [4]:
! unzip -o FFSC_MOU_v2_IndustryOnly_20250401.zip

Archive:  FFSC_MOU_v2_IndustryOnly_20250401.zip
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000001.freelist  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000001.gdbindexes  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000001.gdbtable  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000001.gdbtablx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000001.TablesByName.atx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000002.gdbtable  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000002.gdbtablx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000003.gdbindexes  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000003.gdbtable  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000003.gdbtablx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000004.CatItemsByPhysicalName.atx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000004.CatItemsByType.atx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000004.FDO_UUID.atx  
  inflating: FFSC_MOU_v2_IndustryOnly.gdb/a00000004.freelist  
  inflating: FFSC_MOU_v2_I

## **Enriching Timber Industry Data**

Converts the Timber Industry actual treatments dataset into the Task Force standardized schema. Dataset is enriched with vegetation, ownership, county, WUI, Task Force Region, and year. 

In [5]:

import warnings
import logging
import time
from datetime import datetime
import psutil
import os
from enrich.enrich_Timber_Industry import enrich_Timber_Industry

# Get the current process ID
process = psutil.Process(os.getpid())

a_reference_gdb_path = "a_Reference.gdb"
start_year, end_year = 1950, 2025
output_gdb_path = f"/tmp/USFS_{start_year}_{end_year}.gdb"

ti_input_gdb_path = "FFSC_MOU_v2_IndustryOnly.gdb"
ti_input_layer_name = "FFSC_MOU_IndustryOnly_Pol"
   
output_gdb_path = f"/tmp/Timber_Industry_Spatial_{start_year}_{end_year}.gdb"
output_layer_name = f"Timber_Industry_Spatial_{datetime.today().strftime('%Y%m%d')}"
    
enrich_Timber_Industry(ti_input_gdb_path,
                       ti_input_layer_name,
                       a_reference_gdb_path,
                       start_year,
                       end_year,
                       output_gdb_path,
                       output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-12-08 20:05:46,437 INFO  [enrich.Timber_Industry]  Load the Timeber Industry Spatial Layer into a GeoDataFrame
2025-12-08 20:05:46,794 INFO  [enrich.Timber_Industry]     time for loading FFSC_MOU_IndustryOnly_Pol: 0.3553507328033447
2025-12-08 20:05:46,970 INFO  [enrich.Timber_Industry]  Performing Standardization...
2025-12-08 20:05:46,970 INFO  [enrich.Timber_Industry]     step 1/15 Clip Features to California...
2025-12-08 20:06:19,785 INFO  [enrich.Timber_Industry]        time for loading California and clipping: 32.81390929222107
2025-12-08 20:06:19,787 INFO  [enrich.Timber_Industry]     step 2/15 Repairing Geometry...
2025-12-08 20:06:20,365 INFO  [enrich.Timber_Industry]     step 3/15 Adding Common Columns...
2025-12-08 20:06:20,445 INFO  [enrich.Timber_Industry]     step 4/15 Transfering Values...


KeyError: 'Organization'